In [24]:
import pandas as pd
import numpy as np
import polars as pl

import matplotlib.pyplot as plt
import seaborn as sns

sns.set_style()
plt.rcParams["figure.figsize"] = (12, 6)

En este cuadernillo se quiere calcular el ranking de todos los recintos hospitalarios en GRD para cada diagnóstico principal a lo largo de los años. Se utilizará la base de datos de egresos hospitalarios GRD.

In [25]:
df_grd = pl.read_csv(
    "../data/processed/df_procesada.csv",
    separator=";",
    columns=["ESTANCIA", "COD_HOSPITAL", "DIAGNOSTICO1", "ANIO_EGRESO"],
)

df_grd = df_grd.with_columns(
    pl.col("DIAGNOSTICO1").str.replace(".", "", literal=True).str.pad_end(4, "X")
)

In [27]:
# Calcula el conteo de egresos y el ranking por diagnostico y anio
ranking_egresos_nacionales_grd = (
    df_grd.group_by(pl.col(["ANIO_EGRESO", "COD_HOSPITAL", "DIAGNOSTICO1"]))
    .agg([pl.len().alias("n_egresos"), pl.col("ESTANCIA").sum().alias("dias_estada_totales")])
    .sort(by=["ANIO_EGRESO", "DIAGNOSTICO1", "n_egresos"], descending=True)
    .with_columns(
        pl.col("n_egresos")
        .rank(method="min", descending=True)
        .over(["ANIO_EGRESO", "DIAGNOSTICO1"])
        .alias("ranking_egresos")
    )
)

# Guarda el ranking de egresos
ranking_egresos_nacionales_grd.write_csv(
    "../data/interim/ranking_nacionales_egresos_grd.csv", separator=";"
)